In [ ]:
import os, sys, import_ipynb, ipynb
if os.getcwd() == 'C:\\Users\\admin\\Desktop\\retail_data_analysis\\code':
    print ("Already in code directory")
else:
    os.chdir('..//code')
        
# import python libraries
from importLibraries import *


def fun_h2oClassification():
    h2o.init()
    os.chdir(os.path.expanduser('../input'))
    
    # Import training data 
    rawdata_marketplace = pd.read_csv('input_data_preProcessed.csv')
    
    rawdata_marketplace = rawdata_marketplace.head(1000)
    
    rawdata_marketplace = rawdata_marketplace[['UnitPrice', 'CustomerID', 'Country', 'new_Only_InvoiceDate', 'new_Only_InvoiceHour', 'revenue', 'noun_1', 'nearestConcept']]
    
    na_list = ['NA', 'none', 'nan', 'NaT']
    col_dtypes = {'UnitPrice':'numeric', 'CustomerID':'string', 'Country': 'string', 'new_Only_InvoiceDate': 'string', 'new_Only_InvoiceHour': 'string', 'revenue': 'numeric', 'noun_1': 'string', 'nearestConcept': 'string'}
    rawdata_marketplace_h2o = h2o.H2OFrame(rawdata_marketplace, column_types= col_dtypes, na_strings=na_list)
    
    #display(rawdata_marketplace.describe())
    
    # Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each    
    rawdata_marketplace_h2o_train,rawdata_marketplace_h2o_test,rawdata_marketplace_h2o_valid = rawdata_marketplace_h2o.split_frame(ratios=[.7, .15])
    display(rawdata_marketplace_h2o_train.shape, rawdata_marketplace_h2o_test.shape, rawdata_marketplace_h2o_valid.shape)
    
    predictors = rawdata_marketplace_h2o_train.columns
    response = "nearestConcept"
    predictors.remove(response)
    
    rawdata_marketplace_h2o_train[response] = rawdata_marketplace_h2o_train[response].asfactor()
    rawdata_marketplace_h2o_test[response] = rawdata_marketplace_h2o_test[response].asfactor()
    rawdata_marketplace_h2o_valid[response] = rawdata_marketplace_h2o_valid[response].asfactor()
    
    
    # Run AutoML for n base models (limited to 1 hour max runtime by default)
    aml = H2OAutoML(max_models=4)
    aml.train(x=predictors, y=response, training_frame=rawdata_marketplace_h2o_train)

    # AutoML Leaderboard
    lb = aml.leaderboard

    # Optionally edd extra model information to the leaderboard
    lb = get_leaderboard(aml, extra_columns='ALL')

    # Print all rows
    lb.head(rows=lb.nrows)
    
    # The leader model is stored here
    display(aml.leader)
    
    preds = aml.predict(rawdata_marketplace_h2o_test)
    # or:
    #preds = aml.leader.predict(test)
    display(preds.head())

    # Stacked Ensemble “All Models” model is an ensemble of all of the individual models which is often the top-performing model on the leaderboard.
    # Get the model ids for all models in the Leaderboard
    model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
    # Get the "All Models" Stacked Ensemble model
    se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
    # Get the Stacked Ensemble metalearner model
    metalearner = h2o.get_model(se.metalearner()['name'])
    
    #display(metalearner)
    #metalearner.std_coef_plot()
    
    os.chdir('..//output')
    #  save and download the model 
    h2o.save_model(aml.leader, path="my_model_saved")
    
    # Get a particular model
    metalearner = h2o.get_model(lb[1, 'model_id'])
    
    # Get the data in pandas df
    metalearner.varimp(True).to_csv('h2oClassifierFeatureimportances.csv')
    
    